In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-23"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11131,2024-02-23,Eua Nba,00:00,Sacramento Kings,San Antonio Spurs,1.20,4.94,241.5,1.85,2.09,-10.5,1.97,10.50,MJ0eM3Ds,0.833333,0.202429,0.540541,0.478469,0.035762,0.6,0.2,NaN,NaN,249.160,71.970336,0.288852,444.294,126.923555,0.285675,172.412,444.294,240.30,608.22,0.0,1.0,0.0,0.0,0.861427,0.086145,0.967381,-1.89,-0.378,-0.529101,0.700195,0.5,-0.200195,-1.32,-0.264,-14.924242,0.283342,0.2,-0.083342
11132,2024-02-23,Eua Nba,21:30,Philadelphia 76ers,Cleveland Cavaliers,2.69,1.51,228.5,1.82,1.99,3.5,1.99,1.35,4rj0LNSm,0.371747,0.662252,0.549451,0.502513,0.033999,0.4,0.2,NaN,NaN,264.126,139.677512,0.528829,154.520,24.945664,0.161440,207.624,242.392,181.44,153.51,0.0,0.0,0.0,0.0,0.397327,0.063101,0.270987,-5.00,-1.000,-1.690000,0.620505,0.4,-0.220505,2.06,0.412,1.237864,0.518750,0.7,0.181250
11133,2024-02-23,Eua Nba,22:00,Houston Rockets,Phoenix Suns,2.21,1.71,228.5,1.86,1.94,2.5,2.05,1.46,Iof4Ksrf,0.452489,0.584795,0.537634,0.515464,0.037284,0.4,0.6,NaN,NaN,264.456,68.932720,0.260659,191.118,48.678406,0.254703,272.758,205.230,291.90,256.51,0.0,0.0,0.0,0.0,0.180384,0.029773,0.237717,-0.83,-0.166,-7.289157,0.533875,0.6,0.066125,-2.30,-0.460,-1.543478,0.607025,0.5,-0.107025
11134,2024-02-23,Eua Nba,22:00,New Orleans Pelicans,Miami Heat,1.71,2.21,223.5,1.85,1.95,-3.5,2.01,2.69,A18DGasJ,0.584795,0.452489,0.540541,0.512821,0.037284,0.6,0.2,NaN,NaN,192.510,102.858540,0.534302,276.976,125.141041,0.451812,173.240,268.966,147.63,262.69,0.0,0.0,0.0,0.0,0.180384,0.037216,0.204610,-1.30,-0.260,-2.730769,0.713772,0.7,-0.013772,2.48,0.496,2.439516,0.413981,0.7,0.286019
11135,2024-02-23,Mundo Americup,21:10,República Dominicana,México,1.36,2.97,157.5,1.81,1.89,-6.5,1.83,6.50,tWq4DanP,0.735294,0.336700,0.552486,0.529101,0.071994,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.525839,0.030578,0.792842,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11136,2024-02-23,Mundo Americup,21:10,Uruguai,Panamá,1.30,3.34,143.5,1.80,1.88,-12.5,1.88,13.00,CzUqgAqD,0.769231,0.299401,0.555556,0.531915,0.068632,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.621766,0.030744,1.056859,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11137,2024-02-23,Ásia Copa Asiática,12:00,Iraque,Arábia Saudita,1.99,1.74,138.5,1.83,1.83,-2.5,1.83,1.93,jyvsNzle,0.502513,0.574713,0.546448,0.546448,0.077225,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.094786,0.000000,0.037612,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11138,2024-02-23,Eua Ncaa,00:00,CS Fullerton,Cal Poly,1.19,4.91,130.5,1.80,1.95,-10.5,1.91,11.00,CUW7zhRh,0.840336,0.203666,0.555556,0.512821,0.044002,0.4,0.6,NaN,NaN,176.562,110.725057,0.627117,374.186,164.071718,0.438476,126.754,454.282,146.00,598.40,0.0,1.0,0.0,0.0,0.862438,0.056569,0.995755,-3.45,-0.690,-0.275362,0.000000,0.0,0.000000,-5.00,-1.000,-3.910000,0.196555,0.1,-0.096555
11139,2024-02-23,Eua Ncaa,00:00,CS Northridge,UC Davis,2.05,1.80,149.5,1.80,1.95,1.5,1.91,1.63,vHYaxEdt,0.487805,0.555556,0.555556,0.512821,0.043360,0.6,0.6,NaN,NaN,159.368,66.416939,0.416752,124.070,48.349804,0.389698,121.106,165.382,159.14,149.80,0.0,0.0,0.0,0.0,0.091832,0.056569,0.111859,-2.02,-0.404,-2.599010,0.000000,0.0,0.000000,0.43,0.086,9.302326,0.477855,0.5,0.022145
11140,2024-02-23,Eua Ncaa,00:00,UC Riverside,UC San Diego,2.33,1.65,1

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Sacramento Kings,San Antonio Spurs,241.5,1.85,1.0,Over
1,22:00,Eua Nba,New Orleans Pelicans,Miami Heat,223.5,1.85,1.0,Over
2,21:10,Mundo Americup,República Dominicana,México,157.5,1.81,1.0,Over
3,21:10,Mundo Americup,Uruguai,Panamá,143.5,1.80,1.0,Over
4,00:00,Eua Ncaa,CS Fullerton,Cal Poly,130.5,1.80,1.0,Over
5,01:00,Eua Ncaa,Arizona,Washington State,150.5,1.91,1.0,Over
6,01:00,Eua Ncaa,Santa Clara,Loyola Marymount,145.5,1.91,1.0,Over
7,01:00,Eua Ncaa,Stanford,Oregon,150.5,1.91,1.0,Over
8,01:00,Eua Ncaa,UC Irvine,UC Santa Barbara,142.5,1.80,1.0,Over
9,21:00,Eua Ncaa,Canisius,Niagara,141.5,1.83,1.0,Over
